In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

Socrata Open Data API (SODA) 

In [2]:
#Read in csv
df=pd.read_csv('https://data.cityofnewyork.us/resource/43nn-pn8j.csv?$limit=500000')

In [3]:
df=pd.DataFrame(df)

In [4]:
df.groupby("boro").head(10)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,...,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,40980062,TOKYO SUSHI JAPANESE RESTAURANT,Staten Island,1710,VICTORY BOULEVARD,10314.0,7187271771,Japanese,2016-06-28T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.613213,-74.120626,501.0,49.0,16901.0,5017876.0,5.007070e+09,SI07
1,50073403,EL TEQUILAZO RESTAURANT & BAR,Queens,17237,HILLSIDE AVE,11432.0,6462584663,Mexican,2018-08-01T00:00:00.000,Establishment Closed by DOHMH. Violations wer...,...,2021-01-24T06:01:49.000,Pre-permit (Operational) / Initial Inspection,40.712174,-73.788954,408.0,24.0,45800.0,4210593.0,4.098440e+09,QN06
2,50005601,CONGEE VILLAGE,Manhattan,100,ALLEN STREET,10002.0,2129411818,Chinese,2016-12-27T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Initial Inspection,40.718673,-73.990631,103.0,1.0,1800.0,1005472.0,1.004140e+09,MN27
3,50033207,ISTANBUL BAY TURKISH CAFE,Brooklyn,2202,86 STREET,11214.0,7187144300,Turkish,2017-10-18T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.601844,-73.993666,311.0,47.0,29600.0,3167094.0,3.063810e+09,BK28
4,50012650,LOS POBLANOS RESTAURANT,Brooklyn,4312,3 AVENUE,11232.0,3477630571,Mexican,2018-01-18T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Initial Inspection,40.651470,-74.010657,307.0,38.0,200.0,3010901.0,3.007270e+09,BK32
5,50069103,MAISON KAYSER,Manhattan,575,LEXINGTON AVENUE,10022.0,2125206700,Bakery,2017-10-13T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Pre-permit (Non-operational) / Initial Inspection,40.757209,-73.971827,106.0,4.0,10000.0,1036461.0,1.013060e+09,MN19
6,41164678,NIOS RESTAURANT,Manhattan,130,WEST 46 STREET,10036.0,2124852400,American,2020-02-13T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.757807,-73.983280,105.0,4.0,11900.0,1022645.0,1.009980e+09,MN17
7,50098223,MERYENDA,Manhattan,230,PARK AVENUE,10169.0,4152602740,Other,1900-01-01T00:00:00.000,NaN,...,2021-01-24T06:02:05.000,NaN,40.754501,-73.975950,105.0,4.0,9200.0,1036185.0,1.013000e+09,MN19
8,41185792,MR. CHOW TRIBECCA,Manhattan,121,HUDSON STREET,10013.0,2129659500,Chinese,2019-01-22T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Initial Inspection,40.720045,-74.008575,101.0,1.0,3900.0,1002127.0,1.001888e+09,MN24
9,40995117,GOLDEN KRUST CARIBBEAN BAKERY & GRILL,Bronx,4708,WHITE PLAINS ROAD,10470.0,7183250900,Caribbean,2018-08-09T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Calorie Posting / Initial Inspection,40.903703,-73.850146,212.0,11.0,44200.0,2071837.0,2.051120e+09,BX62


In [5]:
# checking missing data in data 
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
grade_date,199033,50.066661
grade,196718,49.484323
score,18495,4.652409
critical_flag,10922,2.747424
violation_description,10922,2.747424


In [6]:
#remove any rows that have no grade, no business name (dba), or location information
df = df.dropna(subset=['grade','dba','latitude','longitude','zipcode','street'])

In [7]:
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
grade_date,2286,1.164087
critical_flag,1668,0.849387
violation_description,1668,0.849387
bin,912,0.464413
violation_code,911,0.463904


## now data is good without NaN



In [8]:
df.columns = df.columns.str.replace("CUISINE DESCRIPTION","CUISINE")
df.columns = df.columns.str.replace(" ","_")# remove space 

In [9]:
df["dba"].replace("'","", inplace=True) # Remove apostrophe
df["dba"].replace(" ?\(.+\)", "", regex=True, inplace=True) # Remove values in parenthesis
df["dba"].replace(" ?#.*", "", regex=True, inplace=True) # Remove # followed by some string


In [10]:
len(df)

196377

In [11]:
clean_df=df# make a copy of the clean dataframe, then don't need to run over again 

In [12]:
clean_df.grade.replace(['A', 'B','C','N','P','Z'], ['ABC','BC','C','Z','Z','Z'], inplace=True) # replace ABC with 123
clean_df.head(5)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,...,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,40980062,TOKYO SUSHI JAPANESE RESTAURANT,Staten Island,1710,VICTORY BOULEVARD,10314.0,7187271771,Japanese,2016-06-28T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.613213,-74.120626,501.0,49.0,16901.0,5017876.0,5.007070e+09,SI07
3,50033207,ISTANBUL BAY TURKISH CAFE,Brooklyn,2202,86 STREET,11214.0,7187144300,Turkish,2017-10-18T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.601844,-73.993666,311.0,47.0,29600.0,3167094.0,3.063810e+09,BK28
6,41164678,NIOS RESTAURANT,Manhattan,130,WEST 46 STREET,10036.0,2124852400,American,2020-02-13T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.757807,-73.983280,105.0,4.0,11900.0,1022645.0,1.009980e+09,MN17
10,41577301,COPPELIA CUBAN LUNCHEONETTE,Manhattan,207,WEST 14 STREET,10011.0,2128585001,"Latin (Cuban, Dominican, Puerto Rican, South &...",2019-04-04T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Initial Inspection,40.738784,-74.000224,104.0,3.0,8100.0,1013729.0,1.007640e+09,MN13
12,50009665,POCKET BAR NYC,Manhattan,455,WEST 48 STREET,10036.0,6466829062,American,2017-10-10T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.762877,-73.991380,104.0,3.0,12700.0,1026579.0,1.010580e+09,MN15


In [13]:
top20=clean_df.groupby(['cuisine_description']).count().sort_values(by='dba', ascending=False).head(20)

In [14]:
top20list=top20.index
top20list=['American', 'Chinese', 'Café/Coffee/Tea', 'Pizza', 'Mexican', 'Italian',
       'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
       'Japanese', 'Bakery', 'Caribbean', 'Spanish', 'Pizza/Italian',
       'Chicken', 'Donuts', 'Indian', 'Asian', 'Delicatessen', 'Jewish/Kosher',
       'Hamburgers', 'Thai']

In [15]:
df1=clean_df[clean_df['cuisine_description'].isin(top20list)] # filter, select cusine under top20
df1=pd.DataFrame(df1) #make sure it's a dataframe
df1.head()
 

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,...,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,40980062,TOKYO SUSHI JAPANESE RESTAURANT,Staten Island,1710,VICTORY BOULEVARD,10314.0,7187271771,Japanese,2016-06-28T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.613213,-74.120626,501.0,49.0,16901.0,5017876.0,5.007070e+09,SI07
6,41164678,NIOS RESTAURANT,Manhattan,130,WEST 46 STREET,10036.0,2124852400,American,2020-02-13T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.757807,-73.983280,105.0,4.0,11900.0,1022645.0,1.009980e+09,MN17
10,41577301,COPPELIA CUBAN LUNCHEONETTE,Manhattan,207,WEST 14 STREET,10011.0,2128585001,"Latin (Cuban, Dominican, Puerto Rican, South &...",2019-04-04T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Initial Inspection,40.738784,-74.000224,104.0,3.0,8100.0,1013729.0,1.007640e+09,MN13
12,50009665,POCKET BAR NYC,Manhattan,455,WEST 48 STREET,10036.0,6466829062,American,2017-10-10T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.762877,-73.991380,104.0,3.0,12700.0,1026579.0,1.010580e+09,MN15
13,50075705,LA TRANQUILITE RESTAURANT INC,Brooklyn,9128,AVENUE L,11236.0,7185318767,Caribbean,2020-03-07T00:00:00.000,Violations were cited in the following area(s).,...,2021-01-24T06:01:49.000,Cycle Inspection / Re-inspection,40.635978,-73.899441,318.0,46.0,100800.0,3232574.0,3.082560e+09,BK50


In [16]:
len(df1)# 198892 vs 160942

158787

Moved the conversion from algorithm part to here

In [17]:
#convert longitude and latitude to float to do precise calculations
df1["longitude"] = df1["longitude"].astype(float)
df1["latitude"] = df1["latitude"].astype(float)

# User Input

In [18]:
#install ipywidgets package
!pip install ipywidgets  

You should consider upgrading via the '/Users/valerie/anaconda3/bin/python -m pip install --upgrade pip' command.


In [19]:
#install ipywidgets package
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [0]:
#install ipyleaflet package
!pip install ipyleaflet

In [20]:
import ipywidgets as widgets #Package for python widgets
from ipyleaflet import Map,basemaps,Marker #Package enabling interactive maps in the Jupyter notebook
from IPython.display import display

In [21]:
#Get the User Coordinates by defining for now

my_latitude = 40.728453 #float(input("Type latitude:"))
my_longitude = -73.979612 #float(input("Type longitude:"))


In [22]:
#Initialize a 5*2 grid layout
grid = widgets.GridspecLayout(5, 2, height='900px')

#Initialize a Map
center = [my_latitude, my_longitude]
zoom = 20
m = Map(basemap=basemaps.OpenStreetMap.Mapnik,center=center, zoom=zoom)

#Initialize a dropdown box, and make the options to be the top20list 
dropdown_cuisine = widgets.Dropdown(options = top20list, description = 'Cuisine Type:')

#Initialize a radio buttons for grade selection
radio_grade = widgets.RadioButtons(options=[('A', 'ABC'), ('B or above', 'BC'), ('C or above', 'C')], \
                                   description='Grade:', \
                                   disabled=False)

#Initialize an output widget 
output = widgets.Output()

#Add each component to the grid layout
grid[0,0] = dropdown_cuisine
grid[0,1] = radio_grade
grid[1,:] = m
grid[2:4,:] = output

In [23]:
#Define the event handler for dropdown_cuisine
def dropdown_cuisine_eventhandler(change):
    common_filtering(change.new, radio_grade.value, m) #change.new is the lastest value from dropdown_cuisine

#Define the event handler for radio_grade
def radio_grade_eventhandler(change):
    common_filtering(dropdown_cuisine.value, change.new, m) #change.new is the lastest value from radio_grade
    
#Define the event handler for map
def map_eventhandler(change):
    common_filtering(dropdown_cuisine.value, radio_grade.value, change.value) #change.new is the lastest value from map


#Define the filtering function (from Karim's algorithm) 
def common_filtering(cuisine, grade, m):
    #clear the previous selection output on each iteration
    output.clear_output()
    
    #clear the previous 5 markers on the map
    base_layer = m.layers[0]
    m.clear_layers()
    m.add_layer(base_layer)
    
    #Assign the values based on user's choice
    restaurant_choice = cuisine
    grade_choice = grade
    
    #ALGORITHM STARTS HERE#
    #restaurant have multiple times they were test, keep only last grade
    filtered_df = df1.sort_values('grade_date').drop_duplicates('dba',keep='last') 

    #filter by restaurant choices
    filtered_df=filtered_df.loc[df1['cuisine_description'] == restaurant_choice] 

    #filter by grade of interest
    filtered_df=filtered_df.loc[df1['grade'] == grade_choice]

    #calculate proximity: use absolute value delta of user's lat / long and the restaurant's; convert to miles where 1 degree = 68.703 miles
    filtered_df["distance"] = (abs(filtered_df['latitude'] - my_latitude) + abs(filtered_df['longitude'] - my_longitude)) * 68.703

    # replace grades back the grades that are understandable
    filtered_df.grade.replace(['ABC','BC','C'], ['A', 'B','C'], inplace=True)

    #clean up what columns to keep for results
    results_df = filtered_df.loc[:,["dba","cuisine_description","building","street","boro","grade","grade_date","latitude","longitude","distance"]].copy()

    #sort by total distance column, smallest at the top
    results_df = results_df.sort_values(["distance"],ascending=[True])

    #get the top 5 results to display
    results_df = results_df.head(5)
    
    for index, row in results_df.iterrows():        
        center = (row['latitude'],row['longitude'])
        marker = Marker(location = center, draggable = False, title = row['dba'])
        m.add_layer(marker)
    
    #display the result dataframe
    with output:
        display(results_df)

In [24]:
#bind the event handlers to the dropdown/radio selection and map by the observe() function
#so that every time user makes new selections, it will invoke dropdown_cuisine_eventhandler/radio_grade_eventhandler/map_eventhandler
dropdown_cuisine.observe(dropdown_cuisine_eventhandler, names='value')
radio_grade.observe(radio_grade_eventhandler, names='value')
m.observe(map_eventhandler, names='value')


The below will display a great map in Jupyter notebook.

In [25]:
#Display the grid layout
display(grid)

GridspecLayout(children=(Dropdown(description='Cuisine Type:', layout=Layout(grid_area='widget001'), options=(…

In [27]:
restaurant_choice = input("Type 1 for Italian or 2 for Chinese:")  #fake a user interface for now
if restaurant_choice == "1":
    restaurant_choice = "Italian"
else:
    restaurant_choice = "Chinese"  
grade_choice = "BC" #say that I just want B grades
my_latitude = 40.728453 #float(input("Type latitude:"))
my_longitude = -73.979612 #float(input("Type longitude:"))

Type 1 for Italian or 2 for Chinese:1


# Algorithm

In [28]:
#convert longitude and latitude to float to do precise calculations
#df1["longitude"] = df1["longitude"].astype(float)
#df1["latitude"] = df1["latitude"].astype(float)

#restaurant have multiple times they were test, keep only last grade
filtered_df = df1.sort_values('grade_date').drop_duplicates('dba',keep='last') 

#filter by restaurant choices
filtered_df=filtered_df.loc[df1['cuisine_description'] == restaurant_choice] 

#filter by grade of interest
filtered_df=filtered_df.loc[df1['grade'] == grade_choice]

#calculate proximity: use absolute value delta of user's lat / long and the restaurant's; convert to miles where 1 degree = 68.703 miles
filtered_df["distance"] = (abs(filtered_df['latitude'] - my_latitude) + abs(filtered_df['longitude'] - my_longitude)) * 68.703

# replace grades back the grades that are understandable
filtered_df.grade.replace(['ABC','BC','C'], ['A', 'B','C'], inplace=True)

#clean up what columns to keep for results
results_df = filtered_df.loc[:,["dba","cuisine_description","building","street","boro","grade","grade_date","latitude","longitude","distance"]].copy()

#sort by total distance column, smallest at the top
results_df = results_df.sort_values(["distance"],ascending=[True])

#get the top 5 results to display
results_df = results_df.head(5)
results_df

,dba,cuisine_description,building,street,boro,grade,grade_date,latitude,longitude,distance
273955,JOE & PAT'S,Italian,168,1 AVENUE,Manhattan,B,2019-06-18T00:00:00.000,40.728954,-73.984280,0.355109
50202,TABLESIDE ITALIAN COOK SHOPPE,Italian,345,EAST 6 STREET,Manhattan,B,2020-02-28T00:00:00.000,40.726937,-73.987430,0.641279
25421,SUPPER,Italian,156,EAST 2 STREET,Manhattan,B,2019-10-22T00:00:00.000,40.722767,-73.985180,0.773130
259490,IL BUCO RESTAURANT,Italian,47,BOND STREET,Manhattan,B,2020-02-06T00:00:00.000,40.726021,-73.992885,1.079024
245697,LENA'S ITALIAN KITCHEN,Italian,551,2 AVENUE,Manhattan,B,2020-01-15T00:00:00.000,40.742528,-73.977474,1.113906


# Export to CSV

In [0]:
from google.colab import files
results_df.to_csv('results.csv') 
files.download('results.csv')

### above is a sample dataframe 

In [0]:
GradeA=len(df.loc[df["grade"]=="A"])

GradeB=len(df.loc[df["grade"]=="B"])

GradeC=len(df.loc[df["grade"]=="C"])
GradeN=len(df.loc[df["grade"]=="N"])
GradeP=len(df.loc[df["grade"]=="p"])
GradeZ=len(df.loc[df["grade"]=="Z"])

In [0]:
Grades=[GradeA,GradeB,GradeC,GradeN,GradeP,GradeZ]


In [0]:
Grade_count=pd.DataFrame(Grades,columns = ['Grade'], index=['A', 'B', 'C',"N",'P','Z'])

Grade_count.plot.bar()

In [0]:
boro_counts=clean_df.boro.value_counts()
boro_counts.plot.bar()
plt.title('Cuisine count', fontsize = 15)
plt.xlabel('Cuisine', fontsize = 12)

In [0]:
gb_all = clean_df.groupby("grade").grade.agg(["count"])
gb_all["percentage"] = gb_all / gb_all.sum() * 100

gb_recent = clean_df.drop_duplicates("KEY").groupby("grade").grade.agg(["count"])
gb_recent["percentage"] = gb_recent / gb_recent.sum() * 100

gb_all = gb_all.join(gb_recent,lsuffix=" all", rsuffix=" most recent")#

gb_all[["percentage all", "percentage most recent"]].plot(kind="bar")
gb_all.style.format("{:,.2f}")
#N = Not Yet Graded
#Z = Grade Pending P= Grade Pending issued on re-opening following an initial inspection that resulted in a closure

In [0]:
most_common_cuisine = clean_df.groupby(["KEY"])["cuisine_description"].agg(lambda x: x.value_counts().index[0])



In [0]:
most_common_cuisine 

In [0]:
common_cuisine=most_common_cuisine.tolist()
Counter(common_cuisine).most_common(40)


In [0]:
#Functions

#Get input from User （python widgets?)
#def 

#Filter all qualified restaurants by constraints
#def restaurantsFilter(currentLat, currentLong, constraints[])